In [1]:
import logging
import os
import numpy as np
import pandas as pd

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.experiment.PcitExperiment import PcitExperiment, _Clustering
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.experiment.CepsExperiment import CepsExperiment
pd.options.display.max_columns = 999

In [2]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=5,
    seed=0,
)

In [ ]:
t_c = 50
t_v = 100
K = 2
n = 2
t_ini = sum([i * train_instances.size * 10 for i in range(1, K + 1)]) * 2
t_i = 100
max_iter = 1
solver_class = TSP_LKH_Solver
instance_class = TSP_Instance


experiment = CepsExperiment(
    t_c=t_c,
    t_v=t_v,
    t_ini=t_ini,
    t_i=t_i,
    K=K,
    n=n,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-11-17 20:50:28] INFO      [CEPS] Start!
[2024-11-17 20:50:28] INFO      Initialization
[2024-11-17 20:50:28] DEBUG     {'ASCENT_CANDIDATES': 52, 'BACKBONE_TRIALS': 1, 'BACKTRACKING': 'YES', 'CANDIDATE_SET_TYPE': 'ALPHA', 'EXTRA_CANDIDATES': 7, 'EXTRA_CANDIDATE_SET_TYPE': 'QUADRANT', 'GAIN23': 'NO', 'GAIN_CRITERION': 'YES', 'INITIAL_STEP_SIZE': 1, 'INITIAL_TOUR_ALGORITHM': 'QUICK-BORUVKA', 'INITIAL_TOUR_FRACTION': 0.5820197920751, 'KICKS': 3, 'KICK_TYPE': 0, 'MAX_BREADTH': 954863081, 'MAX_CANDIDATES': 10, 'MOVE_TYPE': 3, 'PATCHING_A': 3, 'PATCHING_C': 1, 'POPULATION_SIZE': 8, 'RESTRICTED_SEARCH': 'YES', 'SUBGRADIENT': 'YES', 'SUBSEQUENT_MOVE_TYPE': 6, 'SUBSEQUENT_PATCHING': 'NO'}
[2024-11-17 20:50:28] DEBUG     {'ASCENT_CANDIDATES': 45, 'BACKBONE_TRIALS': 0, 'BACKTRACKING': 'YES', 'CANDIDATE_SET_TYPE': 'DELAUNAY', 'EXTRA_CANDIDATES': 10, 'EXTRA_CANDIDATE_SET_TYPE': 'QUADRANT', 'GAIN23': 'YES', 'GAIN_CRITERION': 'YES', 'INITIAL_STEP_SIZE': 3, 'INITIAL_TOUR_ALGORITHM': 'SIERPINSKI',

In [24]:
import random
random.choice(range(best_portfolio.size))

0

In [ ]:
# t_c = 30
# t_v = 30
# K = 2
# n = 2
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = GlobalExperiment(
#     t_c=t_c,
#     t_v=t_v,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 30
# t_v = 30
# K = 2
# n = 3
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = ParhydraExperiment(
#     t_c=t_c,
#     t_v=t_v,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
t_c = 50
t_v = 100
K = 2
n = 2
max_iter = 2
solver_class = TSP_LKH_Solver
instance_class = TSP_Instance

experiment = PcitExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    max_iter=max_iter,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# remaining_time = np.ones(shape=(K, )) * np.inf
# best_portfolio.evaluate(test_instances, remaining_time, "test")

In [ ]:
from src.database import db_connect
conn = db_connect()

In [26]:
import sqlite3
# conn = sqlite3.connect("_archive/3/PARHYDRA.db")
conn = sqlite3.connect("database/2024_11_17_19_32_27.db")

In [27]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,INITIAL_TOUR_ALGORITHM,INITIAL_TOUR_FRACTION,KICKS,KICK_TYPE,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,2220588739511820280,52,1,YES,ALPHA,7,QUADRANT,NO,YES,1,QUICK-BORUVKA,0.582020,3,0,954863081,10,3,3,1,8,YES,YES,6,NO
1,350772795015577985,45,1,NO,DELAUNAY,0,QUADRANT,NO,NO,3,NEAREST-NEIGHBOR,0.284280,1,5,47289226,10,3,2,5,21,NO,YES,0,NO
2,690612294992071927,51,0,NO,DELAUNAY,5,QUADRANT,NO,YES,1,GREEDY,0.679016,4,5,1889571907,9,3,2,1,61,YES,YES,2,NO
3,956979786962138268,40,0,YES,QUADRANT,8,QUADRANT,NO,YES,1,WALK,0.166626,5,4,414934836,8,2,1,2,85,YES,NO,5,YES
4,2051815471081033189,45,0,YES,DELAUNAY,10,QUADRANT,YES,YES,3,SIERPINSKI,0.842342,3,5,1010150627,9,6,3,1,26,YES,NO,4,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,1814373750366912683,49,1,NO,DELAUNAY,5,QUADRANT,YES,YES,5,WALK,0.283465,4,4,1619267412,5,2,5,3,79,YES,NO,2,NO
280,1461809342729900636,44,1,NO,NEAREST-NEIGHBOR,2,QUADRANT,NO,NO,1,SIERPINSKI,0.766393,2,5,1301946407,3,4,4,1,86,YES,NO,5,YES
281,1874694739534421708,44,1,NO,NEAREST-NEIGHBOR,8,QUADRANT,NO,YES,4,GREEDY,0.838828,3,4,623373234,8,3,3,2,48,NO,NO,0,NO
282,163769879414466589,43,0,YES,ALPHA,3,QUADRANT,NO,NO,3,BORUVKA,0.845419,0,4,312150098,6,6,2,4,16,YES,NO,6,NO


In [28]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id,angle_min,angle_median,angle_mean,angle_max,angle_sd,angle_span,angle_coef_of_var,centroid_centroid_x,centroid_centroid_y,centroid_dist_min,centroid_dist_median,centroid_dist_mean,centroid_dist_max,centroid_dist_sd,centroid_dist_span,centroid_dist_coef_of_var,cluster_01pct_number_of_clusters,cluster_01pct_mean_distance_to_centroid,cluster_05pct_number_of_clusters,cluster_05pct_mean_distance_to_centroid,cluster_10pct_number_of_clusters,cluster_10pct_mean_distance_to_centroid,bounding_box_10_ratio_of_cities_outside_box,bounding_box_20_ratio_of_cities_outside_box,bounding_box_30_ratio_of_cities_outside_box,chull_area,chull_points_on_hull,distance_distances_shorter_mean_distance,distance_distinct_distances,distance_mode_frequency,distance_mode_quantity,distance_mode_mean,distance_mean_tour_length,distance_sum_of_lowest_edge_values,distance_min,distance_median,distance_mean,distance_max,distance_sd,distance_span,distance_coef_of_var,modes_number,mst_depth_min,mst_depth_median,mst_depth_mean,mst_depth_max,mst_depth_sd,mst_depth_span,mst_depth_coef_of_var,mst_dists_min,mst_dists_median,mst_dists_mean,mst_dists_max,mst_dists_sd,mst_dists_span,mst_dists_coef_of_var,mst_dists_sum,nnds_min,nnds_median,nnds_mean,nnds_max,nnds_sd,nnds_span,nnds_coef_of_var,num_nodes,cost_matrix_avg,cost_matrix_std,cost_matrix_skew,stdTime,mst_length,mst_length_avg,mst_length_std,mst_length_skew,mst_degree_avg,mst_degree_std,mst_degree_skew,mstTime,cluster_distance_avg,cluster_distance_std,cluster_distance_skew,clusterTime,tour_const_heu_avg,tour_const_std,tour_const_skew,ls_impov_per_step_avg,ls_impov_per_step_std,ls_impov_per_step_skew,ls_steps_2lm_avg,ls_steps_2lm_std,ls_steps_2lm_skew,ls_maxdist_avg,ls_maxdist_std,ls_maxdist_skew,ls_bestsol_avg,ls_bestsol_std,ls_bestsol_skew,ls_backbone_avg,ls_backbone_std,ls_backbone_skew,lpTime,bc_improv_per_cut_avg,bc_improv_per_cut_std,bc_improv_per_cut_skew,bc_upper_lower_ratio,bc_no1s_min,bc_no1s_q25,bc_no1s_q50,bc_no1s_q75,bc_no1s_max,bc_p1s,bc_pn1s,bcTime,acc,acfTime
0,data/TSP/CEPS_benchmark/rotation/45.tsp,0.002679,1.533287,1.523353,3.133102,0.929012,3.130423,0.609847,0.551103,0.455445,0.013460,0.335516,0.320439,0.679751,0.140566,0.666291,0.438669,2.0,0.299215,1.0,0.320439,1.0,0.320439,0.23625,0.48375,0.74875,0.814314,0.02375,0.525484,1.0,800.0,0.000002,0.0,710.877319,12.204579,0.0,0.429124,0.443743,1.292313,0.218912,1.292313,0.493330,1.0,1.0,6.0,16.03250,81.0,20.879389,80.0,1.302316,0.000705,0.019883,0.020435,0.065852,0.010498,0.065147,0.513744,0.000057,0.000705,0.014163,0.015662,0.047361,0.009126,0.046656,0.582704,800.0,383591.198008,0.521889,1.862155,0.03,0.044028,0.044083,0.509745,2.328925,1.9975,0.340460,5.882461,0.01,0.085990,0.139633,-2.429917,1.02,0.057564,0.000000e+00,0.000000e+00,0.003310,0.000000,-3.000000e-09,0.112500,0.000000,0.000000,0.16375,0.000000,0.000000,39.648629,0.000000,0.000000,1.000000,0.000000,0.000000,0.96,0.002396,1.925998,0.593799,0.009510,0.100000,0.5,0.5,0.5,0.900000,0.90375,0.19500,0.11,338.112966,0.68
1,data/TSP/CEPS_benchmark/expansion/09.tsp,0.002400,1.494609,1.499733,3.137329,0.878209,3.134930,0.585577,0.592388,0.525414,0.007522,0.221080,0.228677,0.652673,0.119554,0.645150,0.522807,8.0,0.136453,2.0,0.156762,2.0,0.157269,0.01125,0.19000,0.67250,0.569420,0.01125,0.536903,1.0,800.0,0.000002,0.0,506.589155,6.185907,0.0,0.291779,0.316222,1.068205,0.182046,1.068205,0.575688,2.0,1.0,6.0,16.70375,84.0,21.668188,83.0,1.297205,0.000222,0.009908,0.012129,0.336555,0.015752,0.336333,1.298669,0.000048,0.000222,0.007122,0.009066,0.105043,0.009328,0.104821,1.028873,800.0,312921.294181,0.575117,2.691012,0.02,0.038286,0.038334,1.300459,0.079203,1.9975,0.333957,7.742967,0.02,0.361774,1.263227,1.134118,1.29,0.053927,0.000000e+00,0.000000e+00,0.007079,0.001024,1.173474e+00,0.107750,0.007284,-2.928246,0.16925,0.010065,-2.643985,36.235461,0.023421,0.733827,0.870511,0.253229,-0.559499,2.15,0.011224,3.518124,0.880483,0.012699,0.083333,0.5,0.5,0.5,0.916667,0.90750,0.18875,0.21,401.661792,0.69
2,da

In [29]:
df = pd.read_sql_query("SELECT * FROM results", conn)
# df.to_excel("tmp.xlsx")
df

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/rotation/45.tsp,2220588739511820280,100.0,10.0,configuration,2024-11-17 18:32:51
1,data/TSP/CEPS_benchmark/expansion/09.tsp,2220588739511820280,100.0,10.0,configuration,2024-11-17 18:32:51
2,data/TSP/CEPS_benchmark/explosion/41.tsp,2220588739511820280,100.0,10.0,configuration,2024-11-17 18:32:51
3,data/TSP/CEPS_benchmark/linearprojection/40.tsp,2220588739511820280,100.0,10.0,configuration,2024-11-17 18:32:51
4,data/TSP/CEPS_benchmark/uniform_portgen/18.tsp,2220588739511820280,100.0,10.0,configuration,2024-11-17 18:32:52
...,...,...,...,...,...,...
4130,data/TSP/CEPS_benchmark/cluster/47.tsp,374085570969732237,100.0,10.0,test,2024-11-17 20:10:58
4131,data/TSP/CEPS_benchmark/expansion/22.tsp,1537059346176113225,100.0,10.0,test,2024-11-17 20:10:58
4132,data/TSP/CEPS_benchmark/expansion/22.tsp,1032217821252663560,100.0,10.0,test,2024-11-17 20:11:00
4133,data/TSP/CEPS_benchmark/expansion/22.tsp,530109802942539602,100.0,10.0,test,2024-11-17 20:11:00


In [30]:
df.loc[df["comment"] == "configuration"]

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/rotation/45.tsp,2220588739511820280,100.0,10.0,configuration,2024-11-17 18:32:51
1,data/TSP/CEPS_benchmark/expansion/09.tsp,2220588739511820280,100.0,10.0,configuration,2024-11-17 18:32:51
2,data/TSP/CEPS_benchmark/explosion/41.tsp,2220588739511820280,100.0,10.0,configuration,2024-11-17 18:32:51
3,data/TSP/CEPS_benchmark/linearprojection/40.tsp,2220588739511820280,100.0,10.0,configuration,2024-11-17 18:32:51
4,data/TSP/CEPS_benchmark/uniform_portgen/18.tsp,2220588739511820280,100.0,10.0,configuration,2024-11-17 18:32:52
...,...,...,...,...,...,...
2166,data/TSP/CEPS_benchmark/uniform_portgen/39.tsp,2244918699606022259,100.0,10.0,configuration,2024-11-17 19:51:25
2167,data/TSP/CEPS_benchmark/linearprojection/40.tsp,2244918699606022259,100.0,10.0,configuration,2024-11-17 19:51:25
2168,data/TSP/CEPS_benchmark/grid/39.tsp,2244918699606022259,100.0,10.0,configuration,2024-11-17 19:51:25
2169,data/TSP/CEPS_benchmark/linearprojection/05.tsp,2244918699606022259,100.0,10.0,configuration,2024-11-17 19:51:26


In [31]:
_ = df.loc[df["comment"] == "test"].pivot_table(index="instance_id", columns="solver_id", values="cost")
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_

solver_id,1032217821252663560,1537059346176113225,374085570969732237,530109802942539602,best,min
instance_id,,,,,,
data/TSP/CEPS_benchmark/cluster/00.tsp,7.54,1.39,1.70,3.17,1,1.39
data/TSP/CEPS_benchmark/cluster/01.tsp,100.00,2.97,6.56,5.77,1,2.97
data/TSP/CEPS_benchmark/cluster/02.tsp,100.00,100.00,100.00,100.00,0,100.00
data/TSP/CEPS_benchmark/cluster/03.tsp,1.08,100.00,8.75,0.58,3,0.58
data/TSP/CEPS_benchmark/cluster/04.tsp,100.00,100.00,100.00,100.00,0,100.00
...,...,...,...,...,...,...
data/TSP/CEPS_benchmark/uniform_portgen/45.tsp,0.91,7.72,1.97,0.95,0,0.91
data/TSP/CEPS_benchmark/uniform_portgen/46.tsp,0.93,100.00,0.60,100.00,2,0.60
data/TSP/CEPS_benchmark/uniform_portgen/47.tsp,0.60,1.22,2.19,1.98,0,0.60


In [32]:
_["min"].describe()

count    470.000000
mean       3.991383
std       14.972481
min        0.110000
25%        0.632500
50%        1.140000
75%        2.177500
max      100.000000
Name: min, dtype: float64

In [33]:
(_["min"] > 90).value_counts()

False    459
True      11
Name: min, dtype: int64

In [ ]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

In [ ]:
_.sort_values(by="min")

In [ ]:
# config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
# config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
# solver = TSP_LKH_Solver(config=config)

# instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

# solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index